# Генерация текста с использованием Word2Vec

Метод Word2Vec помогает компьютеру представлять слова как числа, чтобы он мог работать с ними так же, как мы работаем с числами в математике.

Принцип работы:

1. Мы представляем каждое слово как набор чисел. Например, слово "книга" может быть представлено как список из 100 чисел, скажем, [0.2, -0.3, 0.5, ...].
2. Мы показываем компьютеру много предложений, чтобы он понял, какие слова часто встречаются рядом друг с другом. Например, слова "собака" и "лай" часто встречаются вместе, а слова "собака" и "школа" — реже.
3. Компьютер создает специальную модель, которая запоминает эти связи. Эта модель превращает каждое слово в список чисел таким образом, что слова с похожим значением имеют похожие списки чисел. Например, слова "собака" и "щенок" будут иметь очень похожие списки чисел, потому что они часто используются в похожих ситуациях.

Пример

Представим, что мы дали компьютеру много предложений, например:

- "Кошка сидит на диване."
- "Собака лает на кошку."
- "Кошка и собака спят."

После обучения компьютер может понять, что "кошка" и "собака" связаны с "диван", "лает" и "спят". Если мы попросим компьютер найти похожие слова к слову "кошка", он может предложить слово "собака", потому что они часто встречаются в похожих предложениях.

In [1]:
import os

dir_export = 'export'
os.makedirs(dir_export, exist_ok=True)

In [6]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 10.1 MB/s eta 0:00:0000:010:01m


### Импорт необходимых библиотек

In [2]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

### Вставка отрывка из текста

In [3]:
text = """
Ну, князь, Генуя и Лукка стали не больше как поместьями семейства Бонапарте. Нет, я предупреждаю, если вы не скажете, что война неизбежна, если вы все еще будете защищать все гадости, все ужасы этого Антихриста (уж я верю в это), я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как говорите.
"""

### Предобработка текста и токенизация

In [4]:
# Удаление пунктуации и приведение к нижнему регистру
text = re.sub(r'[^\w\s]', '', text.lower())

# Токенизация
tokens = word_tokenize(text, language='russian')

# Удаление стоп-слов
stop_words = set(stopwords.words('russian'))
tokens = [word for word in tokens if word not in stop_words]

print(tokens)

['князь', 'генуя', 'лукка', 'стали', 'поместьями', 'семейства', 'бонапарте', 'предупреждаю', 'скажете', 'война', 'неизбежна', 'будете', 'защищать', 'гадости', 'ужасы', 'антихриста', 'верю', 'это', 'знаю', 'друг', 'верный', 'раб', 'говорите']


### Обучение модели Word2Vec

In [8]:
from gensim.models import Word2Vec

# Обучение модели Word2Vec
model = Word2Vec([tokens], vector_size=100, window=5, min_count=1, workers=4)

# Сохранение модели для последующего использования
model_filename = os.path.join(dir_export, "word2vec_russian_model")
model.save(model_filename)

### Генерация текста

Предсказывание следующего слова на основе текущего контекста

In [9]:
import random

# Функция для генерации текста
def generate_text(model, seed_text, max_length=20):
    result = seed_text.copy()
    for _ in range(max_length):
        # Получаем векторы для текущих слов
        word_vectors = [model.wv[word] for word in result if word in model.wv]
        if not word_vectors:
            break
        # Средний вектор
        mean_vector = sum(word_vectors) / len(word_vectors)
        # Находим наиболее похожие слова
        similar_words = model.wv.similar_by_vector(mean_vector, topn=10)
        # Выбираем случайное слово из топ-10 наиболее похожих
        next_word = random.choice(similar_words)[0]
        result.append(next_word)
    return ' '.join(result)

# Начальное слово
seed_text = ['князь']

# Генерация текста
generated_text = generate_text(model, seed_text)
print(generated_text)

князь верный раб стали семейства стали генуя защищать генуя поместьями это князь князь защищать семейства раб верный защищать стали генуя князь
